In [1]:
import requests

In [2]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [3]:
documents = []

for course in documents_raw:
    course_name = course['course']
    
    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [4]:
import minsearch

index = minsearch.Index(
    text_fields = ['question', 'text', 'section'],
    keyword_fields = ['course']
)

index.fit(documents)

In [14]:
# Set the OPENAI_API_KEY before running this cell
# import os
# os.environ
# os.environ['OPENAI_API_KEY']

In [6]:
from openai import OpenAI

openai_client = OpenAI()

In [7]:
def search(query):
    boost={'question': 3.0, 'section':0.5}
    
    results = index.search(
        query=query,
        filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    
    return results

In [8]:
def build_prompt(query, search_results):
    prompt_template="""
    You are a course teaching assistant.
    Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONETXT:
    {context}
    """.strip()
    
    context=""
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt

In [9]:
def llm(prompt):
    
    response = openai_client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{'role': 'user', 'content': prompt}]
    )
    
    return response.choices[0].message.content

In [10]:
def rag(query):
    
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    
    return answer

In [11]:
rag('how do I run Kafka?')

"To run Kafka using Java, navigate to your project directory and execute the following command in the terminal:\n\n```\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nFor running a Python Kafka producer, ensure you create a virtual environment and install the required dependencies by following these steps:\n\n1. Create a virtual environment and install packages (run only once):\n   ```\n   python -m venv env\n   source env/bin/activate  # For MacOS/Linux\n   pip install -r ../requirements.txt\n   ```\n\n2. To activate the virtual environment (you'll need to run this every time you require it):\n   ```\n   source env/bin/activate  # For MacOS/Linux\n   ```\n   (For Windows, the command is `env\\Scripts\\activate`)\n\n3. To deactivate the virtual environment:\n   ```\n   deactivate\n   ``` \n\nMake sure Docker images are running before executing your Python files."

In [12]:
rag('the course has already started, can I still enroll?')

"Yes, you can still enroll in the course even after it has started. You are eligible to submit homework assignments, but do keep in mind that there will be deadlines for turning in the final projects. Therefore, it's important not to leave everything until the last minute."

# RAG with Vector Search

In [15]:
from qdrant_client import QdrantClient, models

In [16]:
qd_client = QdrantClient("http://localhost:6333/")

In [17]:
EMBEDDING_DIMS = 512

model_handle = 'jinaai/jina-embeddings-v2-small-en'

collection_name = 'zoomcamp-faq'

In [18]:
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMS,
        distance=models.Distance.COSINE
    )
)

True

In [20]:
points = []

for idx, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector=models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=idx,
        vector=vector,
        payload=doc
    )
    points.append(point)

points[0]

PointStruct(id=0, vector=Document(text="Course - When will the course start? The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", model='jinaai/jina-embeddings-v2-small-en', options=None), payload={'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with ann

In [21]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [22]:
question = 'I just discovered the course. Can I still join it?'

In [27]:
def vector_search(question):
    course='data-engineering-zoomcamp'

    query_points = qd_client.query_points(
        collection_name = collection_name,
        query=models.Document(
            text=question,
            model=model_handle
        ),
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=5,
        with_payload=True
    )

    results = []
    
    for pont in query_points.points:
        results.append(point.payload)
    
    return results

In [28]:
def vector_rag(query):
    search_results = vector_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [29]:
rag('how do I run kafka?')

'To run Kafka, you can execute the following command in the terminal within your project directory:\n\n```\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nMake sure to replace `<jar_name>` with the actual name of your JAR file.'